In [1]:
import pandas as pd
from tqdm import tqdm
import time
#todo   return all fields needed for the prediction

In [2]:
#!-- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- !#
print("\nInitialized..")
#!-- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- !#

with tqdm(total=100) as pbar:

    #*   Prohibits truncatation of data in prints.
    pd.set_option('display.max_colwidth', -1)

    #*   User input of which movie to munge attributes for then predict against the model.
    movieToSearch = str(input("Enter a movie title: "))  #"Toy Story 4"
    pbar.update(5)

    #*   The `title.basics` db object:
    #*   Declares $tconst (IMDB's media PK) as a string & $genres as a string array.
    print("\n\n\nMunging title.basics.tsv..")
    herp = pd.read_csv("/Users/nicolespaar/Documents/title.basics.tsv", sep='\t', header=0, low_memory=False)
    pbar.update(10)
    herpArray = herp[herp['primaryTitle'] == movieToSearch]
    herpRow = herpArray[herpArray['titleType'] == 'movie']
    tconst = herpRow.tconst.to_string(index=False)
    genres = herpRow.genres.to_string(index=False).split(",")

    #*   The `title.crew` db object:
    #*   Declares $crew_directors & $crew_writers as a string array.
    print("\nMunging title.crew.tsv..")
    derp = pd.read_csv("/Users/nicolespaar/Documents/title.crew.tsv", sep='\t', header=0)
    pbar.update(10)
    derpRow = derp[derp['tconst'] == tconst[1:]]
    crew_directors = derpRow.directors.to_string(index=False).split(",")
    crew_writers = derpRow.writers.to_string(index=False).split(",")

    #*   The `name.basics` db object:
    print("\nMunging name.basics.tsv..")
    merp = pd.read_csv("/Users/nicolespaar/Documents/name.basics.tsv", sep='\t', header=0, index_col='nconst')
    pbar.update(10)

    #*   The `title.principals` db object:
    print("\nMunging title.principals.tsv..")
    plerp = pd.read_csv("/Users/nicolespaar/Documents/title.principals.tsv", sep='\t', header=0, index_col='nconst')
    pbar.update(10)
    plerpRow = plerp[plerp['tconst'] == tconst[1:]]

  0%|          | 0/100 [00:00<?, ?it/s]


Initialized..
Enter a movie title: Toy Story 4


  5%|▌         | 5/100 [00:04<01:34,  1.01it/s]




Munging title.basics.tsv..


 15%|█▌        | 15/100 [00:22<01:44,  1.23s/it]

...

 25%|██▌       | 25/100 [00:29<01:19,  1.06s/it]


Munging name.basics.tsv..


 35%|███▌      | 35/100 [00:51<01:31,  1.41s/it]


Munging title.principals.tsv..


 45%|████▌     | 45/100 [01:39<02:13,  2.42s/it]


In [4]:
    #!-- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- !#
    print("\nSorting Cast..")
    time.sleep(0.25)
    #!-- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- !#
    #*   Iterrows() on $plerpRow to sort/bin the cast actors' names & jobs from the cast non-actors'
    #*   names and jobs.

    cast_actors = []
    cast_actors_jobs = []
    cast_other = []
    cast_other_jobs = []

    for row in plerpRow.iterrows():
        x = row[1].category
        if (x in ('actor', 'actress')):
            cast_actors.append(row[0])
            cast_actors_jobs.append(row[1].job)
        else:
            cast_other.append(row[0])
            cast_other_jobs.append(row[1].job)

    pbar.update(10)

    def lengths(x):
        if isinstance(x, list):
            yield len(x)
            for y in x:
                yield from lengths(y)

    def iAmount(anArray):
        return max(lengths(anArray))
    #todo   a check to panic if c_a vs c_a_j (and c_o vs c_o_j) are not the same length


Sorting Cast..


In [5]:
    #!-- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- !#
    print("\nPrinting Cast..\n")
    time.sleep(0.25)
    #!-- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- !#
    #*   A for-loop executing as many times as the length of the $cast_actors list prints out all of
    #*   the relevant information. Also plugs in the current iterration's actor's `nconst` into the
    #*   $merp db object (where `nconst` is the Index) to return a whole row/object relating to them.

    print("\n*****   Primary Cast[ACTORS]")
    i = 0
    iAmt = iAmount(cast_actors)

    for i in range(iAmt):
        cast = cast_actors[i]
        job = cast_actors_jobs[i]
        print("IMDB 'nconst': ", cast, "\n", "Job: ", job, "\n\n", "Cast Object: ", "\n", merp.loc[cast], "\n")
        i += 1

    pbar.update(5)

    #*   A for-loop executing as many times as the length of the $cast_other list prints out all of
    #*   the relevant information. Also plugs in the current iterration's actor's `nconst` into the
    #*   $merp db object (where `nconst` is the Index) to return a whole row/object relating to them.

    print("\n*****   Primary Cast[NON-ACTORS]")
    i = 0
    iAmt = iAmount(cast_other)

    for i in range(iAmt):
        cast = cast_other[i]
        job = cast_other_jobs[i]
        print("IMDB 'nconst': ", cast, "\n", "Job: ", job, "\n\n", "Cast Object: ", "\n", merp.loc[cast], "\n")
        i += 1

    pbar.update(5)


Printing Cast..


*****   Primary Cast[ACTORS]
IMDB 'nconst':  nm0000158 
 Job:  \N 

 Cast Object:  
 primaryName          Tom Hanks                              
birthYear            1956                                   
deathYear            \N                                     
primaryProfession    producer,actor,soundtrack              
knownForTitles       tt0109830,tt0162222,tt0120815,tt0094737
Name: nm0000158, dtype: object 

IMDB 'nconst':  nm0000741 
 Job:  \N 

 Cast Object:  
 primaryName          Tim Allen                              
birthYear            1953                                   
deathYear            \N                                     
primaryProfession    actor,producer,miscellaneous           
knownForTitles       tt0101120,tt0177789,tt0114709,tt0111070
Name: nm0000741, dtype: object 

IMDB 'nconst':  nm0001633 
 Job:  \N 

 Cast Object:  
 primaryName          Annie Potts                            
birthYear            1952                      

In [6]:
    #!-- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- !#
    print("\nPrinting Crew..")
    time.sleep(0.25)
    #!-- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- !#
    #*   A for-loop on $crew_directors (the 2 columns of the $merp db object) to
    #*   print out all of the relevant information. Also plugs in the current iterration's crew's
    #*   `nconst` to return a whole row/object relating to them.

    print("\n*****   All Crew[DIRECTORS]")

    for crew in crew_directors:
        if crew[0] == " ":
            print("\n", "        [!]      [!]      [!]      [!]      [!]")
            print(f"[!] Prepending whitespace bug detected on: '{crew}'", "[!]", "\n")
            crew = crew[1:]
            print(f"[!]             Fixed to: '{crew}'", "                  [!]")
            print("        [!]      [!]      [!]      [!]      [!]", "\n")
        print("IMDB 'nconst': ", crew, "\n", "Job: ", job, "\n\n", "Cast Object: ", "\n", merp.loc[crew], "\n")

    pbar.update(5)



    #*   A for-loop on $crew_directors (the 2 columns of the $merp db object) to
    #*   print out all of the relevant information. Also plugs in the current iterration's crew's
    #*   `nconst` to return a whole row/object relating to them.

    print("\n*****   All Crew[WRITERS]")

    for crew in crew_writers:
        if crew[0] == " ":
            print("\n", "        [!]      [!]      [!]      [!]      [!]")
            print(f"[!] Prepending whitespace bug detected on: '{crew}'", "[!]", "\n")
            crew = crew[1:]
            print(f"[!]             Fixed to: '{crew}'", "                  [!]")
            print("        [!]      [!]      [!]      [!]      [!]", "\n")
        print("IMDB 'nconst': ", crew, "\n", "Job: ", job, "\n\n", "Cast Object: ", "\n", merp.loc[crew], "\n")

    pbar.update(5)


Printing Crew..

*****   All Crew[DIRECTORS]

         [!]      [!]      [!]      [!]      [!]
[!] Prepending whitespace bug detected on: ' nm2155757' [!] 

[!]             Fixed to: 'nm2155757'                   [!]
        [!]      [!]      [!]      [!]      [!] 

IMDB 'nconst':  nm2155757 
 Job:  original story by 

 Cast Object:  
 primaryName          Josh Cooley                            
birthYear            \N                                     
deathYear            \N                                     
primaryProfession    actor,art_department,writer            
knownForTitles       tt0382932,tt2096673,tt1049413,tt1979376
Name: nm2155757, dtype: object 


*****   All Crew[WRITERS]

         [!]      [!]      [!]      [!]      [!]
[!] Prepending whitespace bug detected on: ' nm0005124' [!] 

[!]             Fixed to: 'nm0005124'                   [!]
        [!]      [!]      [!]      [!]      [!] 

IMDB 'nconst':  nm0005124 
 Job:  original story by 

 Cast Object:  
 pri

In [7]:
    #!-- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- !#
    print("\nPrinting Genres..\n")
    time.sleep(0.25)
    #!-- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- !#
    #*   A for-loop on $genres to print out all of the genres listed for the movieToSearch.
    print("\n*****   All Genres")
    for genre in genres:
        if genre[0] == " ":
            print("\n", "        [!]      [!]      [!]      [!]      [!]")
            print(f"[!] Prepending whitespace bug detected on: '{genre}'", "[!]", "\n")
            genre = genre[1:]
            print(f"[!]             Fixed to: '{genre}'", "                  [!]")
            print("        [!]      [!]      [!]      [!]      [!]", "\n")
        print(genre)
    print("\n", genres, "\n")

    pbar.update(10)


Printing Genres..


*****   All Genres

         [!]      [!]      [!]      [!]      [!]
[!] Prepending whitespace bug detected on: ' Adventure' [!] 

[!]             Fixed to: 'Adventure'                   [!]
        [!]      [!]      [!]      [!]      [!] 

Adventure
Animation
Comedy

 [' Adventure', 'Animation', 'Comedy'] 



In [ ]:
    #!-- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- !#
    print("\nPrinting Production Companies..\n")
    time.sleep(0.25)
    #!-- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- !#
    

In [46]:
#?   #   #   #   #   #   #   #   #
df = pd.DataFrame(columns=['winner', 'cast', 'crew', 'genres', 'keywords', 'overview',
                          'production_companies', 'production_countries', 'release_date',
                          'runtime', 'status', 'title', 'Bechdel_test'])

df['winner'] = float(1) #doesn't work
df['cast'] = [{#"cast_id": 18, 
    "character": "Woody", #"credit_id": "52fe4291c3a36847f8028fa1", 
    "gender": float(2), #"id": 11390, 
    "name": "Tom Hanks", "order": float(0)}]
df['crew'] = [{
    "credit_id": "5715157dc3a3684122010c0c", 
    "department": "Art", "gender": float(2), #"id": 244, 
    "job": "Director", "name": "Josh Cooley"
}]
df['genres'] = [[{
    "id": None,
    "name": "Adventure"
}, {
    "id": None,
    "name": "Animation"
}, {
    "id": None,
    "name": "Comedy"
}]]
df['keywords'] = [[{
    "id": None,
    "name": "bo peep character"
}, {
    "id": None,
    "name": "buzz lightyear character"
}, {
    "id": None,
    "name": "woody character"
}, {
    "id": None,
    "name": "fourth part"
}]]
df['overview'] = "When a new toy called \"Forky\" joins Woody and the gang, a road trip alongside old and new friends reveals how big the world can be for a toy."
df['production_companies'] = [[{
    "name": "Pixar Animation Studios",
    "id": None
}, {
    "name": "Walt Disney Pictures",
    "id": None
}]]
df['production_countries'] = [{
    "iso_3166_1": "US",
    "name": "United States of America"
}]
df['release_date'] = "2019-06-21"
df['runtime'] = float(100)
df['status'] = "Released"
df['title'] = "Toy Story 4"
df['Bechdel_test'] = float(1)

df

,winner,cast,crew,genres,keywords,overview,production_companies,production_countries,release_date,runtime,status,title,Bechdel_test
0,NaN,"{'character': 'Woody', 'gender': 2.0, 'name': 'Tom Hanks', 'order': 0.0}","{'credit_id': '5715157dc3a3684122010c0c', 'department': 'Art', 'gender': 2.0, 'job': 'Director', 'name': 'Josh Cooley'}","[{'id': None, 'name': 'Adventure'}, {'id': None, 'name': 'Animation'}, {'id': None, 'name': 'Comedy'}]","[{'id': None, 'name': 'bo peep character'}, {'id': None, 'name': 'buzz lightyear character'}, {'id': None, 'name': 'woody character'}, {'id': None, 'name': 'fourth part'}]","When a new toy called ""Forky"" joins Woody and the gang, a road trip alongside old and new friends reveals how big the world can be for a toy.","[{'name': 'Pixar Animation Studios', 'id': None}, {'name': 'Walt Disney Pictures', 'id': None}]","{'iso_3166_1': 'US', 'name': 'United States of America'}",2019-06-21,100.0,Released,Toy Story 4,1.0


In [216]:
# receive the input from extract then TRANSFORM it to an input ready object
import json, ast, re
import numpy as np
import pandas as pd



pd.set_option("display.max_colwidth", -1)


d = {
  "winner": "1",
  "cast": [
    [
      {
        "name": "Tom Hanks",
        "character": "Woody",
        "gender": "2",
        "order": "0"
      },
      {
        "name": "Tim Allen",
        "character": "Buzz Lightyear",
        "gender": "2",
        "order": "1"
      },
      {
        "name": "Annie Potts",
        "character": "Bo Peep",
        "gender": "0",
        "order": "2"
      },
      {
        "name": "Tony Hole",
        "character": "Forky",
        "gender": "2",
        "order": "3"
      }
    ]
  ],
  "crew": [
    [
      {
        "name": "Josh Cooley",
        "department": "Art",
        "gender": "2",
        "job": "Director"
      },
      {
        "name": "John Lasseter",
        "department": "Writing",
        "gender": "2",
        "job": "Screenplay"
      },
      {
        "name": "Andrew Stanton",
        "department": "Writing",
        "gender": "2",
        "job": "Screenplay"
      },
      {
        "name": "Valerie LaPointe",
        "department": "Writing",
        "gender": "0",
        "job": "Screenplay"
      }
    ]
  ],
  "genres": [
    [
      {
        "name": "Adventure"
      },
      {
        "name": "Animation"
      },
      {
        "name": "Comedy"
      }
    ]
  ],
  "keywords": [
    [
      {
        "name": "bo peep character"
      },
      {
        "name": "buzz lightyear character"
      },
      {
        "name": "woody character"
      },
      {
        "name": "fourth part"
      }
    ]
  ],
  "overview": "When a new toy called \"Forky\" joins Woody and the gang, a road trip alongside old and new friends reveals how big the world can be for a toy.",
  "production_companies": [
    [
      {
        "name": "Pixar Animation Studios"
      },
      {
        "name": "Walt Disney Pictures"
      }
    ]
  ],
  "production_countries": [
    {
      "iso_3166_1": "US",
      "name": "United States of America"
    }
  ],
  "release_date": "2019-06-21",
  "runtime": "100",
  "status": "Released",
  "title": "Toy Story 4",
  "Bechdel_Test": "1"
}

#ast.literal_eval(d)

dff = pd.DataFrame(data=d)
#print(dff)

#! Needs MAJOR help
def feature_engineering(column_name, dff, json_name):

    name = {}
    for item in dff[column_name]:
        #print(item)
        group = str(item)
        group = re.split('[^a-zA-Z]', group)
        group = list(filter(None, group))
        
        cName0 = str(group[1] + " " + group[2])
        #print(cName0)
        cName1 = str(group[8] + " " + group[9])
        #print(cName1)
        cName2 = str(group[16] + " " + group[17])
        #print(cName2)
        cName3 = str(group[24] + " " + group[25])
        #print(cName3)
        
        if cName0 not in name:
            name[cName0] = 1
        else:
            name[cName0] += 1
        if cName1 not in name:
            name[cName1] = 1
        else:
            name[cName1] += 1
        if cName2 not in name:
            name[cName2] = 1
        else:
            name[cName2] += 1
        if cName3 not in name:
            name[cName3] = 1
        else:
            name[cName3] += 1
            
    #print(name)

    final = {}
    index = 0
    for k,v in name.items():
        if v > 1:
            final[k] = index
            index += 1
    np_item = np.zeros((len(df),len(final)))
    item_dict = {}
    row = 0
    for item in dff[column_name]:
        group = str(item)
        group = re.split('[^a-zA-Z]', group)
        group = list(filter(None, group))
        
        cName0 = str(group[1] + " " + group[2])
        print(cName0)
        cName1 = str(group[8] + " " + group[9])
        print(cName1)
        cName2 = str(group[16] + " " + group[17])
        print(cName2)
        cName3 = str(group[24] + " " + group[25])
        print(cName3)
        
        if cName0 not in final:
            np_item[row][cName0] = 1
        else:
            final[cName0] += 1
        if cName1 not in final:
            index = final[cName1]
            np_item[row][index] = 1
        else:
            final[cName1] += 1
        if cName2 not in final:
            index = final[cName2]
            np_item[row][index] = 1
        else:
            final[cName2] += 1
        if cName3 not in final:
            index = final[cName3]
            np_item[row][index] = 1
        else:
            final[cName3] += 1
        row += 1

    dff_item = pd.DataFrame(np_item, columns = list(final.keys()))
    dff_output = pd.concat([dff,dff_item],axis = 1)

    return dff_output

df2 = feature_engineering("cast", dff, "name")
df2 = df2.drop("cast", axis=1)

df2


Tom Hanks
Tim Allen
Annie Potts
Tony Hole


IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [208]:
df2

,winner,crew,genres,keywords,overview,production_companies,production_countries,release_date,runtime,status,title,Bechdel_Test
0,1,"[{'name': 'Josh Cooley', 'department': 'Art', 'gender': '2', 'job': 'Director'}, {'name': 'John Lasseter', 'department': 'Writing', 'gender': '2', 'job': 'Screenplay'}, {'name': 'Andrew Stanton', 'department': 'Writing', 'gender': '2', 'job': 'Screenplay'}, {'name': 'Valerie LaPointe', 'department': 'Writing', 'gender': '0', 'job': 'Screenplay'}]","[{'name': 'Adventure'}, {'name': 'Animation'}, {'name': 'Comedy'}]","[{'name': 'bo peep character'}, {'name': 'buzz lightyear character'}, {'name': 'woody character'}, {'name': 'fourth part'}]","When a new toy called ""Forky"" joins Woody and the gang, a road trip alongside old and new friends reveals how big the world can be for a toy.","[{'name': 'Pixar Animation Studios'}, {'name': 'Walt Disney Pictures'}]","{'iso_3166_1': 'US', 'name': 'United States of America'}",2019-06-21,100,Released,Toy Story 4,1
